In [31]:
import re

In [32]:
filename = './Пушкин Александр. Полное собрание стихотворений - royallib.ru.txt'

In [33]:
with open(filename, 'rb') as file:
    data = file.read().decode('cp1251')

In [82]:
poems = data.split('\r\n\r\n\r\n')
library = []

for poem in poems:
    if '\xa0г' in poem and 'Стихотворения' in poem:
        period = re.sub('([^\W\d]|\s)', '', poem)
        if '–' in period:
            period = period.split('–')
            period = str(int(sum(map(int, period)) / len(period)))
    
    couplets = poem.split('\r\n\r\n\t\t')
    
    clear_name = couplets[0]
    if len(clear_name.split('\r\n\t\t')) > 3:
        library.append(
            {
                'year': period,
                'name': '***',
                'poem': clear_name,
            }
        )
        clear_name = '***'
        
    if clear_name.find('(') != -1:
        clear_name = clear_name[:clear_name.find('(')]
        
    if clear_name.find('[') != -1:
        clear_name = clear_name[:clear_name.find('[')]
        
    if not clear_name.startswith('\r\n') and '*' not in clear_name:
        clear_name = '***'
        
    if len(clear_name.split('\r\n\r\n')) > 3:
        clear_name = '***'
        
    clear_name = re.sub('[\r\n]', '', clear_name)
    
    clear_couplets = []
    for couplet in couplets[1:]:
        ru_words = re.findall(r'[А-я]+',  couplet)
        eng_words = re.findall(r'[A-z]+',  couplet)
        if len(ru_words) > len(eng_words):
            clear_couplets.append(couplet)
        
    if not clear_couplets:
        continue
        
    library.append(
        {
            'year': period,
            'name': clear_name,
            'poem': clear_couplets,
        }
    )

In [83]:
import pandas as pd

In [84]:
a = pd.DataFrame(library)['name'].value_counts(dropna=False).to_dict()

In [85]:
a

{'* * *': 227,
 '***': 102,
 '': 6,
 'Элегия': 6,
 'Эпиграмма': 4,
 'Друзьям': 4,
 '<Из письма к Вяземскому.>': 3,
 'В альбом': 3,
 '<На Каченовского.>': 3,
 'На Пучкову': 2,
 'К Языкову': 2,
 'Пробуждение': 2,
 'I.': 2,
 'Гроб Анакреона': 2,
 'Воспоминание': 2,
 'К Каверину': 2,
 'К ней': 2,
 'Приметы': 2,
 'Певец': 2,
 '<Плетневу.>': 2,
 'Портрет': 2,
 'Роза': 2,
 'К Дельвигу': 2,
 '<Записка к Жуковскому.>': 2,
 '<На Воронцова.>': 2,
 '<В. Ф. Раевскому.>': 2,
 'Князю А. М. Горчакову': 2,
 'Амур и Гименей': 2,
 'Разлука': 2,
 'К***': 2,
 'К ***': 2,
 'Старик': 2,
 '<На Надеждина.>': 2,
 '<На стурдзу.>': 2,
 'Дельвигу': 2,
 'Чедаеву': 2,
 'Воспоминания в Царском селе': 2,
 'Стихи, сочиненные ночью во время бессонницы': 1,
 'Письмо к Лиде': 1,
 '<В альбом Сосницкой.>': 1,
 '<Из письма к Вигелю.>': 1,
 'Ответ анониму': 1,
 'Добрый совет': 1,
 'Имянины': 1,
 'Кавказ': 1,
 'Послание к Галичу': 1,
 'Недокончанная картина': 1,
 '<Баратынскому.>': 1,
 '19 Октября 1827': 1,
 'Мадригал м…ой': 1

In [81]:
re.sub('[\r\n]', '', '\r\nПеснь вторая\r\n\r\nГорькие размышления, сон, спасительная мысль')

'Песнь втораяГорькие размышления, сон, спасительная мысль'

In [ ]:
scp -r rusnak@koala02:/home/rusnak/poetry/cs_poetry ./cs_poetry/

In [93]:
!pwd

/home/rusnak/poetry/cs_poetry/processing/raw_data/ru/pushkin


In [88]:
poems[1]

"* * *\r\n\r\n\t\tDis moi, pourquoi l'Escamoteur\r\n\t\tfist-il siffl&#233; par le parterre?\r\n\t\tH&#233;las! c'est que le pauvre auteur\r\n\t\tL'escamota de Moli&#233;re."

In [89]:
poems[2]

'Перевод\r\n\r\n\t\tСкажите мне, почему «Похититель»\r\n\t\tОсвистан партером?\r\n\t\tУвы, потому что бедный автор\r\n\t\tПохитил его у Мольера.'

In [90]:
poems[3]

"* * *\r\n\r\n\t\tJe chante ce combat, que Toly remporta,\r\n\t\tO&#250; inaint guerrier p&#233;rit, o&#249; Paul se signala,\r\n\t\tNicolas Maturin et la belle Nitouche,\r\n\t\tDont la main fut ie prix d'une horrible escarmouche."

In [91]:
poems[4]

'Перевод\r\n\r\n\t\tПою сей бой, в котором Толи одолел,\r\n\t\tГде погиб не один воин, где Поль отличился,\r\n\t\tНиколая Матюрена и прекрасную Нитуш,\r\n\t\tРука которой была трофеем ужасной стычки.'

In [96]:
poems[6].split('\r\n\r\n\t\t')

['\r\nК Наталье',
 "Pourquoi craindrais-j'e de ie dire?\r\n\t\tC'est Margot qui fixe mon go&#250;t.[1 - Почему мне бояться сказать это?Марго пленила мой вкус.]",
 'Так и мне узнать случилось,\r\n\t\tЧто за птица Купидон;\r\n\t\tСердце страстное пленилось;\r\n\t\tПризнаюсь – и я влюблен!\r\n\t\tПролетело счастья время,\r\n\t\tКак, любви не зная бремя,\r\n\t\tЯ живал да попевал,\r\n\t\tКак в театре и на балах,\r\n\t\tНа гуляньях иль в воксалах\r\n\t\tЛегким зефиром летал;\r\n\t\tКак, смеясь во зло Амуру,\r\n\t\tЯ писал карикатуру\r\n\t\tНа любезный женской пол;\r\n\t\tНо напрасно я смеялся,\r\n\t\tНаконец и сам попался,\r\n\t\tСам, увы! с ума сошел.\r\n\t\tСмехи, вольность – всё под лавку\r\n\t\tИз Катонов я в отставку,\r\n\t\tИ теперь я – Селадон!\r\n\t\tМиловидной жрицы Тальи\r\n\t\tВидел прелести Натальи,\r\n\t\tИ уж в сердце – Купидон!',
 'Так, Наталья! признаюся,\r\n\t\tЯ тобою полонен,\r\n\t\tВ первый раз еще, стыжуся,\r\n\t\tВ женски прелести влюблен.\r\n\t\tЦелый день, как ни вер

In [97]:
couplet = poems[6].split('\r\n\r\n\t\t')

In [98]:
couplet

['\r\nК Наталье',
 "Pourquoi craindrais-j'e de ie dire?\r\n\t\tC'est Margot qui fixe mon go&#250;t.[1 - Почему мне бояться сказать это?Марго пленила мой вкус.]",
 'Так и мне узнать случилось,\r\n\t\tЧто за птица Купидон;\r\n\t\tСердце страстное пленилось;\r\n\t\tПризнаюсь – и я влюблен!\r\n\t\tПролетело счастья время,\r\n\t\tКак, любви не зная бремя,\r\n\t\tЯ живал да попевал,\r\n\t\tКак в театре и на балах,\r\n\t\tНа гуляньях иль в воксалах\r\n\t\tЛегким зефиром летал;\r\n\t\tКак, смеясь во зло Амуру,\r\n\t\tЯ писал карикатуру\r\n\t\tНа любезный женской пол;\r\n\t\tНо напрасно я смеялся,\r\n\t\tНаконец и сам попался,\r\n\t\tСам, увы! с ума сошел.\r\n\t\tСмехи, вольность – всё под лавку\r\n\t\tИз Катонов я в отставку,\r\n\t\tИ теперь я – Селадон!\r\n\t\tМиловидной жрицы Тальи\r\n\t\tВидел прелести Натальи,\r\n\t\tИ уж в сердце – Купидон!',
 'Так, Наталья! признаюся,\r\n\t\tЯ тобою полонен,\r\n\t\tВ первый раз еще, стыжуся,\r\n\t\tВ женски прелести влюблен.\r\n\t\tЦелый день, как ни вер

In [12]:
def pushkin_clean(text):
    text = re.sub(r"(\[[0-9]+\])", "", text) # remove notes
    text = re.sub("<\?>", "", text) # remove <?>
    text = re.sub(r"[\[\]\<\>]+", "", text) # remove all brackets
    text = re.sub("…", ".", text)

    return text.strip().lower()

In [4]:
with open(file) as f:
    lines = f.readlines()

poems = []
poem = []

texts = []
text = ''

el_counter_prev = 0
for i, line in enumerate(lines):
    if line == '\n':
        el_counter = el_counter_prev + 1
    else:
        el_counter = 0

    if el_counter_prev > 3 and el_counter == 0:
        texts.append(pushkin_clean(text))
        text = ''
        
        poems.append(poem)
        poem = []

    if line != '\n':
        if len(text):
            text += " "
        text += line[:-1]
        poem += [pushkin_clean(line[:-1])]

    el_counter_prev = el_counter


In [7]:
poem

['',
 'о нет, мне жизнь не надоела,',
 'я жить люблю, я жить хочу,',
 'душа не вовсе охладела,',
 'утратя молодость свою.',
 'еще хранятся наслажденья',
 'для любопытства моего,',
 'для милых снов воображенья,',
 'для чувств всего.']

In [15]:
with open('./input_as_text.txt', 'w') as f:
    for t in texts:
        f.write(t + "\n")

In [17]:
with open('./input_as_poem.txt', 'w') as f:
    for p in poems:
        for l in p:
            f.write(l + "\n")
        f.write("\n")